In [1]:
import time
from context import remove_far_waters
import mdtraj as md
import numpy as np

In [2]:
sample_trj = 'data/bc_1.xtc'
sample_ref = 'data/md_ref.pdb'

In [3]:
def timer(func_):
    '''Decorator for timing functions.'''
    def function_timer(*args, **kwargs):
        start = time.time()
        result = func_(*args, **kwargs)
        stop = time.time()
        runtime = stop - start
        print(f'Completed {func_.__name__} in runtime of {runtime:10.2f} seconds.')
        
        return result
    return function_timer

In [4]:
t = md.load(sample_trj, top=sample_ref)
print(f'Number of Frames: \t {t.n_frames}')
print(f'Number of Atoms: \t {t.n_atoms}')

Number of Frames: 	 1001
Number of Atoms: 	 40833


### Remove Far Waters

In [34]:
@timer
def init_removewaters():
    t = md.load(sample_trj, top=sample_ref)

    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=100, cutoff=1., verbose=True)
    
init_removewaters()

Trajectory with 1001 frames and 40833 atoms.
36705 water atoms in trajectory.
Completed init_removewaters in runtime of       3.28 seconds.


In [35]:
@timer
def static_search():
    t = md.load(sample_trj, top=sample_ref)
    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=100, cutoff=1., verbose=True)
    t_new = a.static_search()
    
static_search()

Trajectory with 1001 frames and 40833 atoms.
36705 water atoms in trajectory.
Found 47 water molecules within 1.0 nm.
Completed static_search in runtime of       3.69 seconds.


In [5]:
@timer
def dynamic_search():
    t = md.load(sample_trj, top=sample_ref)
    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=100, cutoff=0.5, verbose=True)
    t_new = a.dynamic_search()

dynamic_search()

Trajectory with 1001 frames and 40833 atoms.
36705 water atoms in trajectory.
Distance calculation completed.
Minimum distance to water residues calculated.
Found a maximum of 55 water molecules within a cutoff of 0.5 nm.
Completed dynamic_search in runtime of      87.12 seconds.


In [5]:
@timer
def dynamic_zero():
    t = md.load(sample_trj, top=sample_ref)
    a = remove_far_waters.RemoveWaters(t, sel_query='resid 237', sel='protein', n_waters=10, cutoff=0.5)
    t_new = a.dynamic_zero()

dynamic_zero()

[[[5.5280004 7.5030003 2.4850001]
  [5.525     7.53      2.388    ]
  [5.623     7.498     2.519    ]
  ...
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]]

 [[5.518     7.4990005 2.466    ]
  [5.5540004 7.5380006 2.3790002]
  [5.596     7.4630003 2.516    ]
  ...
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]]

 [[5.505     7.4900002 2.499    ]
  [5.518     7.543     2.413    ]
  [5.598     7.4670005 2.532    ]
  ...
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]]

 ...

 [[5.5010004 7.4860005 2.417    ]
  [5.517     7.537     2.3300002]
  [5.5940003 7.4730005 2.4520001]
  ...
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]]

 [[5.511     7.439     2.4380002]
  [5.577     7.484     2.375    ]
  [5.5680003 7.385     2.5      ]
  ...
  [0.        0.        0.       ]
  [0

In [2]:
t = md.load('data/sample.xtc', top='data/sample.pdb')
t_red = md.load('data/sample_reduced.xtc', top='data/sample_reduced.pdb')

In [3]:
t.xyz.shape

(10, 47301, 3)

In [7]:
md.element.Element.getBySymbol('O')

(8, 'oxygen', 'O', 15.99943, 0.152)

In [8]:
import json

In [16]:
di = {'tip3p':[{'name':'H1', 'element':'H'},
               {'name':'H2', 'element':'H'},
               {'name':'O', 'element':'O'}]}

In [20]:
type(json.loads(json.dumps(di)))

dict

In [34]:
np.concatenate([[a.index for a in r.atoms] for r in t.top.residues if r.is_water]).shape

(43208,)

In [49]:
a = np.array([0.5, 0.1, 0.3, 0.6, 0.2, 0.58])

np.argpartition(a, 3)[:3]

array([4, 2, 1])

In [13]:
a = np.random.rand(5)
np.argsort(a)

array([2, 1, 4, 0, 3])

In [9]:
li = []
[li.append(x) for x in range(4)]
li

[0, 1, 2, 3]

In [5]:
t = md.load(sample_trj, top=sample_ref)
t

<mdtraj.Trajectory with 1001 frames, 40833 atoms, 12530 residues, and unitcells at 0x7fe9c5c8ef28>

In [8]:
water_ids = []
water_res = []
for r in t.top.residues:
    if r.is_water:
        water_res.append(r.index)
        for a in r.atoms:
            water_ids.append(a.index)

In [9]:
water_ids = np.array(water_ids)
water_res = np.array(water_res)
len(water_ids)

36705

In [16]:
import itertools
pairs = itertools.product(range(10), water_ids)
pairs = np.array(list(pairs))

res_of_pairs = np.zeros(len(pairs), dtype=int)
for i_pair, pair in enumerate(pairs):
    a = t.top.atom(pair[1])
    res_of_pairs[i_pair] = a.residue.index

test = [False for x in res_of_pairs]

In [17]:
dist = md.compute_distances(traj=t,
                            atom_pairs=pairs)

In [40]:
start = time.time()
dist_res = np.array([dist[:, res_of_pairs == rid] for rid in water_res])
min_dist_res = np.amin(dist_res, axis=2)
trj_dist_res = min_dist_res.T
stop=time.time()
print(stop - start)

9.344743728637695


In [30]:
dist_res.shape

(12235, 1001, 30)

In [39]:
np.amin(dist_res, axis=2).T.shape

(1001, 12235)

In [31]:
for 

12235